In [1]:
# Install prerequisites

# Install PyCocoTools needed for FasterRCNN
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

# Copy useful functions from pytorch vision tools
%cp ../input/pytorch-vision-tools/references/detection/*.* .

  Cloning https://github.com/gautamchitnis/cocoapi.git (to revision cocodataset-master) to /tmp/pip-req-build-kbal019o
  Running command git clone -q https://github.com/gautamchitnis/cocoapi.git /tmp/pip-req-build-kbal019o
  Running command git checkout -b cocodataset-master --track origin/cocodataset-master
  Switched to a new branch 'cocodataset-master'
  Branch 'cocodataset-master' set up to track remote branch 'cocodataset-master' from 'origin'.
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=272670 sha256=642451f8d1d736654521b3477166564cd908cd4ce5aa6238bfdfc70b09710095
  Stored in directory: /tmp/pip-ephem-wheel-cache-dyhrz36o/wheels/6e/c9/59/56484d4d5ac1ab292a452b4c3870277256551505954fc4a1db
Successfully built pycocotools


In [2]:
# Import required libraries
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import models
from os import path
import torch
import torchvision.utils
import utils
from engine import train_one_epoch, evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import Adam
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pickle
import time
import matplotlib.pyplot as plt
import tqdm
from sklearn.model_selection import train_test_split
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import recall_score, precision_score
import copy

In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
# Load the training data csv file
data_dir = "../input/vinbig1024stratified"

df_train = pd.read_csv(f"{data_dir}/train.csv")
df_val = pd.read_csv(f"{data_dir}/validation.csv")

# Remove images that do not contain any anomolies
#df_train = df_train[df_train["class_id"]!=14]

# Print out the total number of images and the total number of annotations 
print(f"{df_train['image_id'].nunique()} images-train")
print(f"{len(df_train.index)} annotations-train")

print(f"{df_val['image_id'].nunique()} images-val")
print(f"{len(df_val.index)} annotations-val")

11965 images-train
54046 annotations-train
1515 images-val
6867 annotations-val


In [5]:
df_classificaition_train = df_train[["image_id", "class_id"]]
df_classificaition_train["class_id"] = np.where((df_classificaition_train.class_id == 14),0,1)
df_classificaition_train = df_classificaition_train.drop_duplicates()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
df_classificaition_train["class_id"].value_counts()

0    8480
1    3485
Name: class_id, dtype: int64

In [7]:
excess_non_abnormal_imgs = df_classificaition_train["class_id"].value_counts()[0] - df_classificaition_train["class_id"].value_counts()[1]
extra_abnormals_df = df_classificaition_train[df_classificaition_train["class_id"]==1].sample(n=excess_non_abnormal_imgs, replace=True)
df_classificaition_train = df_classificaition_train.append(extra_abnormals_df, ignore_index = True)

In [8]:
df_classificaition_train["class_id"].value_counts()

1    8480
0    8480
Name: class_id, dtype: int64

In [9]:
df_classificaition_val = df_val[["image_id", "class_id"]]
df_classificaition_val["class_id"] = np.where((df_classificaition_val.class_id == 14),0,1)
df_classificaition_val = df_classificaition_val.drop_duplicates()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df_classificaition_val["class_id"].value_counts()

0    1055
1     460
Name: class_id, dtype: int64

In [11]:
# Define a dataset class which defines how to load images,targets for training and validation
class VinBigDataset(Dataset):
    def __init__(self, img_dir, df, transform):
        self.img_dir = img_dir
        self.df = df
        self.imgs = df["image_id"]
        self.transform = transform
        
    def __len__(self):
        # Return the number of elements in the dataset
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # The dataset iterates over each image id
        # Return the requested image,target from the dataset
        
        img_file_name = self.imgs.iloc[idx] + ".png"
        img = np.asarray(Image.open(os.path.join(self.img_dir, img_file_name)).convert("RGB"))
        
        y_label = torch.tensor(float(self.df.iloc[idx, 1]))
        if self.transform is not None:
            img = self.transform(image=img)['image']
            
        img = torch.Tensor(img)
        

        return img, y_label.long()

In [12]:
def create_augmentations(train):
    if train:
        return A.Compose([
                
                A.RandomCrop(height=800,width=800),
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.2, rotate_limit=15),
                A.RandomBrightnessContrast(p=0.4),
                A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
                ToTensorV2(p=1.0)
                    
        ])
    else:
        return A.Compose([
             A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
            
        ])

In [13]:
# Create an instance of the dataset and transformations for training
# and validation
img_folders = "../input/vinbig1024stratified/output/dataset"

# train_img_dir = path.join(data_dir, 'train/train')
train_dataset = VinBigDataset(os.path.join(img_folders, "train"),
                              df_classificaition_train,
                              create_augmentations(train=True))
val_dataset = VinBigDataset(os.path.join(img_folders, "validation"),
                            df_classificaition_val,
                            create_augmentations(train=False))

# Permute the dataset and split into training and validation set 9:1
torch.manual_seed(5262394)
# indices = torch.randperm(len(train_dataset)).tolist()
# val_set_size = len(train_dataset) // 10

# train_dataset = Subset(train_dataset, indices[:-val_set_size])
# val_dataset = Subset(val_dataset, indices[-val_set_size:])

# Create data loaders for the training and validation set. The collate function
# defines how to form a minibatch from the indiviaual data items. In our case we
# just want to collate them into a single list.

data_loader_train = DataLoader(\
    train_dataset, batch_size=5, shuffle=True, num_workers=4)

data_loader_val = DataLoader(\
    val_dataset, batch_size=1, shuffle=False, num_workers=4)

# Print the number of elements in the test and training set
print(f"{len(train_dataset)} items in the training set")
print(f"{len(val_dataset)} items in the validation set")

16960 items in the training set
1515 items in the validation set


In [14]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    val_precision_recall_history = []
    train_acc_history = []
    train_precision_recall_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    lr_scheduler = None
    # set lr schedule
    warmup_factor = 1. / 1000
    warmup_iters = min(1000, len(dataloaders['train']) - 1)

    lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_precision = 0
            running_recall = 0

            # Iterate over data.
            for inputs, labels in tqdm.tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_precision += precision_score(y_true=labels.cpu().detach().numpy(), y_pred=preds.cpu().detach().numpy(),
                                                    zero_division=0)
                running_recall += recall_score(y_true=labels.cpu().detach().numpy(), y_pred=preds.cpu().detach().numpy(),
                                              zero_division=0)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_precision = running_precision / len(dataloaders[phase].dataset)
            epoch_recall = running_recall / len(dataloaders[phase].dataset)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('{} Precision: {:.4f} Recall: {:.4f}'.format(phase, epoch_precision, epoch_recall))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_precision_recall_history.append([epoch_precision, epoch_recall])
            elif phase == 'train':
                train_acc_history.append(epoch_acc)
                train_precision_recall_history.append([epoch_precision, epoch_recall])
        
        if lr_scheduler is not None:
            lr_scheduler.step()
        
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, [train_acc_history, train_precision_recall_history], [val_acc_history, val_precision_recall_history]

In [15]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
            
def initialize_model(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_name = "inception"
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 800

    return model_ft, input_size


# Initialize the model for this run
feature_extract=True
model_ft, input_size = initialize_model(num_classes=2, feature_extract=True, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [16]:
# Send the model to GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=0.001)




Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [17]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

dataloaders_dict = {'train': data_loader_train, 'val': data_loader_val}

# Train and evaluate
model_ft, train_stats, val_stats = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=25, is_inception=True)

  0%|          | 0/3392 [00:00<?, ?it/s]

Epoch 0/24
----------


  0%|          | 0/1515 [00:00<?, ?it/s]

train Loss: 0.9874 Acc: 0.5081
train Precision: 0.1016 Recall: 0.1346


  0%|          | 0/3392 [00:00<?, ?it/s]

...

val Loss: 0.5798 Acc: 0.7063
val Precision: 0.2686 Recall: 0.2686

Epoch 24/24
----------


  0%|          | 0/1515 [00:00<?, ?it/s]

train Loss: 0.7972 Acc: 0.6847
train Precision: 0.1407 Recall: 0.1413


100%|██████████| 1515/1515 [01:25<00:00, 17.71it/s]

val Loss: 0.5637 Acc: 0.7498
val Precision: 0.2693 Recall: 0.2693

Training complete in 285m 33s
Best val Acc: 0.797360


In [18]:
torch.save(model_ft.state_dict(), f"classification_model_weights.bin")
pickle.dump(train_stats, open(f"train_stats.pkl", 'wb'))
pickle.dump(val_stats, open(f"val_stats.pkl", 'wb'))